In [1]:
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

# =========================================
# 1️⃣ Charger CSV
# =========================================
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in dir() else os.getcwd()
csv_path = os.path.join(notebook_dir, "License Plates", "_annotations.csv")
images_folder = os.path.join(notebook_dir, "License Plates")

df = pd.read_csv(csv_path)

# Garder uniquement les plaques
plates_df = df[df["class"] == "license-plate"].copy()

print(f"✓ Total annotations: {len(df)}")
print(f"✓ Plaques trouvées: {len(plates_df)}")

# =========================================
# 2️⃣ SIFT + FLANN
# =========================================
sift = cv2.SIFT_create()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

database = []

# =========================================
# 3️⃣ Construire base (PLAQUE SEULEMENT)
# =========================================
for _, row in plates_df.iterrows():

    filename = row["filename"]
    path = os.path.join(images_folder, filename)

    img = cv2.imread(path)
    if img is None:
        continue

    xmin, ymin, xmax, ymax = map(int, [row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
    plate = img[ymin:ymax, xmin:xmax]

    gray_plate = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray_plate, None)

    if des is not None and len(kp) >= 10:
        database.append({
            "filename": filename,
            "plate": plate,
            "keypoints": kp,
            "descriptors": des
        })

print(f"✓ Base construite avec {len(database)} plaques valides")

# =========================================
# 4️⃣ Charger QUERY
# =========================================
query_filename = "query.png"
query_path = os.path.join(images_folder, query_filename)

query_img = cv2.imread(query_path)

if query_img is None:
    print("❌ Query introuvable")
    exit()

# Chercher annotation dans CSV
query_row = plates_df[plates_df["filename"] == query_filename]

if not query_row.empty:
    print("✓ Annotation trouvée pour la query → découpage automatique")

    row = query_row.iloc[0]
    xmin, ymin, xmax, ymax = map(int, [row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
    query_plate = query_img[ymin:ymax, xmin:xmax]
else:
    print("⚠️ Aucune annotation trouvée → utilisation image complète")
    query_plate = query_img

gray_query = cv2.cvtColor(query_plate, cv2.COLOR_BGR2GRAY)
kp_query, des_query = sift.detectAndCompute(gray_query, None)

if des_query is None:
    print("❌ Aucun descripteur trouvé dans la query")
    exit()

print(f"✓ Query prête: {len(kp_query)} keypoints")

# =========================================
# 5️⃣ Matching avec seuil
# =========================================
MATCH_THRESHOLD = 30
results = []

for item in database:

    matches = flann.knnMatch(des_query, item["descriptors"], k=2)

    good = []
    for pair in matches:
        if len(pair) == 2:
            m, n = pair
            if m.distance < 0.7 * n.distance:
                good.append(m)

    score = len(good)

    if score >= MATCH_THRESHOLD:
        results.append((item, good, score))

# =========================================
# 6️⃣ Affichage résultats
# =========================================
if not results:
    print(f"⚠️ Aucun match >= {MATCH_THRESHOLD}")
else:
    results.sort(key=lambda x: x[2], reverse=True)

    for item, good, score in results:

        print(f"\n✅ MATCH: {item['filename']} | Score: {score}")

        img_matches = cv2.drawMatches(
            query_plate, kp_query,
            item["plate"], item["keypoints"],
            good, None, flags=2
        )

        plt.figure(figsize=(10,6))
        plt.title(f"{item['filename']} | Matches = {score}")
        plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()
    # =========================================
    # 6️⃣ Affichage résultats
    # =========================================
    if not results:
        print(f"⚠️ Aucun match >= {MATCH_THRESHOLD}")
    else:
        results.sort(key=lambda x: x[2], reverse=True)

        for item, good, score in results:

            print(f"\n✅ MATCH: {item['filename']} | Score: {score}")

            img_matches = cv2.drawMatches(
                query_plate, kp_query,
                item["plate"], item["keypoints"],
                good, None, flags=2
            )

            plt.figure(figsize=(10,6))
            plt.title(f"{item['filename']} | Matches = {score}")
            plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
            plt.axis("off")
            plt.show()

✓ Total annotations: 112
✓ Plaques trouvées: 45
✓ Base construite avec 32 plaques valides
❌ Query introuvable
⚠️ Aucune annotation trouvée → utilisation image complète


error: OpenCV(4.13.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 